In [ ]:
"""横排城市化数据-按年度PCA"""

import pandas as pd
import numpy as np
from sklearn.decomposition import PCA

# cats = ['x01','x02','x03','x04','x05','x06','x07','x08','x09','x10']
# catsname = ['城市人口密度','城镇人口比例','城市建成区面积占总面积比例','人均城市道路面积','不透水面面积占总面积比例','人均GDP','第二三产业增加值占GDP比重','人均社会消费品零售总额','万人拥有医疗机构床位','万人拥有医生人数']

results = pd.DataFrame()
pc1 = pd.DataFrame()
percents = pd.DataFrame()

df = pd.read_excel('D:/ee-composite/src/sqlserver/data/U/Data_U_1.xlsx')
# print(df,df.shape)
df1 = df.query('catid=="x01"').sort_values('sn')
for yyyy in range(1985,2021):
    x = df1.loc[:,['sn','cityen','province','provincecn','city','class','classname','classid']]
    x['year'] = yyyy
    results = results.append(x)
results = results.reset_index()   
results.drop('index', axis=1, inplace=True) 
# print(results,results.shape)

for i in range(1,11,1):
    cc = 'x0'+str(i) if i<10 else 'x'+str(i)
    df1 = df.query('catid=="'+cc+'"').sort_values('sn')
    tmp = pd.DataFrame()
    for yyyy in range(1985,2021):
        tmp = tmp.append(df1.loc[:,[yyyy]].rename(columns={yyyy:cc}))
    tmp = tmp.reset_index()[cc]
    tmp = (tmp-tmp.mean())/tmp.std()
    tmp = (tmp-tmp.min())/(tmp.max()-tmp.min())
    results = pd.concat([results,tmp],axis=1)
# print(results,results.shape)

pcadata = (results.loc[:,['sn','cityen','city','province','classname','classid','year','x01','x02','x03','x04','x05','x06','x07','x08','x09','x10']]
            .rename(columns={'city':'citycn','cityen':'city'})
            .set_index(['sn','city','citycn','province','classname','classid','year']))
# print(pcadata,pcadata.shape)

pca = PCA(n_components=1)
tmp = pca.fit_transform(pcadata)
tmp = np.squeeze(tmp)
print(pca.explained_variance_ratio_)

pcadata = pcadata.reset_index()
pcadata['pc1'] = pd.Series(tmp)
pcadata['i'] = pcadata['year']-1984
# print(pcadata,pcadata.shape)

pc1 = pc1.append(pcadata)
pc1x = pc1['pc1']
pc1['pc1'] = (pc1x-pc1x.min())/(pc1x.max()-pc1x.min())
# print(pc1,pc1.shape)

pc1_city_mean = pc1.groupby(['sn','city','citycn','province','classname'])['pc1'].mean()
pc1_city_std = pc1.groupby(['sn','city','citycn','province','classname'])['pc1'].std()
pc1_year_mean = pc1.groupby('year').mean()
pc1_year_std = pc1.groupby('year').std()
# print(pc1_mean,pc1_mean.shape,pc1_std,pc1_std.shape)

pc1['iy'] = pc1['i']*pc1['pc1']
pc1['i2'] = pc1['i']*pc1['i']
pc1_slope = pc1.groupby(['sn','city','citycn','province','classname'])['i','i2','iy','pc1'].sum()
print(pc1_slope,pc1_slope.shape)
pc1_slope['year'] = 2020
pc1_slope['slope'] = (36*pc1_slope['iy']-pc1_slope['i']*pc1_slope['pc1'])/(36*pc1_slope['i2']-pc1_slope['i']*pc1_slope['i'])
print(pc1_slope,pc1_slope.shape)

pc1.to_excel('Data_U_1_PC1.xlsx', sheet_name='PC1')
pc1_city_mean.to_excel('Data_U_1_PC1_city_mean.xlsx', sheet_name='PC1_mean')
pc1_city_std.to_excel('Data_U_1_PC1_city_mean_std.xlsx', sheet_name='PC1_mean_std')
pc1_year_mean.to_excel('Data_U_1_PC1_year_mean.xlsx', sheet_name='PC1_mean')
pc1_year_std.to_excel('Data_U_1_PC1_year_mean_std.xlsx', sheet_name='PC1_mean_std')
pc1_slope.to_excel('Data_U_1_PC1_slope.xlsx', sheet_name='slope')

